## Connect Gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install needed Libs

In [ ]:
!uv pip -q install ultralytics opencv-python matplotlib
import ultralytics
ultralytics.checks()

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.2/112.6 GB disk)


## Add reusable functions

In [ ]:
"""
detect_fish.py

Simple, reusable script to detect fish in images and videos using Ultralytics YOLO (v8+).

Functions:
 - load_model(weights='yolov8n.pt') -> YOLO model
 - detect_image(image_path, model, class_name='fish', conf_thres=0.4, save_path=None)
 - process_video(video_path, model, class_name='fish', conf_thres=0.4, output_path=None)

Notes:
 - This script expects `ultralytics` and `opencv-python` installed.
 - On Colab: install with `pip install ultralytics opencv-python` (and appropriate torch build).
"""

from ultralytics import YOLO
import cv2
import os
import time
import numpy as np
from typing import Tuple, Optional


def load_model(weights: str = 'yolov8n.pt') -> YOLO:
    """Load a YOLO model (from ultralytics).

    Default is `yolov8n.pt` (small, fast). You can switch to `yolov8s.pt`,
    `yolov8m.pt`, or a custom weights file.
    """
    model = YOLO(weights)
    return model


def _draw_box(img: np.ndarray, xyxy: Tuple[int, int, int, int], label: str, conf: float, color=(0,255,0)):
    x1, y1, x2, y2 = xyxy
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    text = f"{label} {conf:.2f}"
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img, (x1, y1 - 18), (x1 + tw, y1), color, -1)
    cv2.putText(img, text, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)


def detect_image(image_path: str, model: YOLO, class_name: str = 'fish', conf_thres: float = 0.4, save_path: Optional[str] = None) -> Tuple[np.ndarray, int]:
    """Detect fish in an image, draw boxes and return annotated image and count.

    - image_path: path to input image
    - model: loaded YOLO model
    - class_name: name of target class to count (e.g., 'fish')
    - conf_thres: confidence threshold for filtering detections
    - save_path: if provided, save annotated image to this path

    Returns (annotated_image, fish_count)
    """
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Could not read image: {image_path}")

    results = model(img)
    res = results[0]

    # model.names maps class index -> name
    names = model.names if hasattr(model, 'names') else {}

    fish_count = 0
    if len(res.boxes) > 0:
        for box in res.boxes:
            # Each box has xyxy, conf, cls as tensors with shape (1,)
            try:
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                conf = float(box.conf[0].cpu().numpy())
                cls_idx = int(box.cls[0].cpu().numpy())
            except Exception:
                # Fallback if API shapes differ
                xyxy = box.xyxy[0].numpy().astype(int)
                conf = float(box.conf[0].numpy())
                cls_idx = int(box.cls[0].numpy())

            label = names.get(cls_idx, str(cls_idx))
            if label.lower() == class_name.lower() and conf >= conf_thres:
                fish_count += 1
                _draw_box(img, tuple(xyxy.tolist()), label, conf, color=(0,200,0))

    # overlay count
    h, w = img.shape[:2]
    info = f"Fish count: {fish_count}"
    cv2.rectangle(img, (5,5), (200,30), (0,0,0), -1)
    cv2.putText(img, info, (10,23), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    if save_path:
        cv2.imwrite(save_path, img)

    return img, fish_count


def process_video(video_path: str, model: YOLO, class_name: str = 'fish', conf_thres: float = 0.4, output_path: Optional[str] = None, max_frames: Optional[int] = None):
    """Process a video frame-by-frame, detect fish, draw boxes and save result video.

    - video_path: input video file
    - model: loaded YOLO model
    - output_path: if None, will create `<video_path>_out.mp4`
    - max_frames: optional cap to process only N frames (useful for tests)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Could not open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    if output_path is None:
        base, ext = os.path.splitext(video_path)
        output_path = base + '_out.mp4'

    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    frame_idx = 0
    t_start = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        res = results[0]

        fish_count = 0
        if len(res.boxes) > 0:
            for box in res.boxes:
                try:
                    xyxy = box.xyxy[0].cpu().numpy().astype(int)
                    conf = float(box.conf[0].cpu().numpy())
                    cls_idx = int(box.cls[0].cpu().numpy())
                except Exception:
                    xyxy = box.xyxy[0].numpy().astype(int)
                    conf = float(box.conf[0].numpy())
                    cls_idx = int(box.cls[0].numpy())

                label = model.names.get(cls_idx, str(cls_idx))
                if label.lower() == class_name.lower() and conf >= conf_thres:
                    fish_count += 1
                    _draw_box(frame, tuple(xyxy.tolist()), label, conf)

        cv2.rectangle(frame, (5,5), (220,30), (0,0,0), -1)
        cv2.putText(frame, f"Fish: {fish_count}", (10,23), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        out.write(frame)

        frame_idx += 1
        if max_frames and frame_idx >= max_frames:
            break

    cap.release()
    out.release()
    t_end = time.time()
    print(f"Processed {frame_idx} frames in {t_end - t_start:.1f}s -> saved to {output_path}")


# if __name__ == '__main__':
#     # Example quick-run using files in this repo's `test-fish` folder
#     here = os.path.dirname(__file__)
#     test_dir = os.path.join(here, 'test-fish')
#     img_example = os.path.join(test_dir, 'Fish-1.jpg')
#     vid_example = os.path.join(test_dir, 'vdo-1.mp4')

#     print('Loading model (this may download weights the first time)')
#     # default to a lightweight YOLOv8 model; change if you need more accuracy
#     model = load_model('yolov8n.pt')

#     if os.path.exists(img_example):
#         out_img = os.path.join(here, 'test-fish', 'Fish-1_out.jpg')
#         _, count = detect_image(img_example, model, class_name='fish', conf_thres=0.4, save_path=out_img)
#         print(f'Image done, fish count={count}, saved {out_img}')

#     if os.path.exists(vid_example):
#         out_vid = os.path.join(here, 'test-fish', 'vdo-1_out.mp4')
#         process_video(vid_example, model, class_name='fish', conf_thres=0.4, output_path=out_vid, max_frames=300)


# Start the prediction

In [ ]:
# Run inference on an image with YOLO11n
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/zidane.jpg'

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 48.4ms
Speed: 12.2ms preprocess, 48.4ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo val model=yolo11n.pt data=coco8.yaml classes=fish

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

WARNING ⚠️ Dataset 'coco8.yaml' images not found, missing path '/content/datasets/coco8/images/val'
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100% ━━━━━━━━━━━━ 25/25 3.6Kfiles/s 0.0s
Dataset download success ✅ (0.5s), saved to /content/datasets

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1773.2±630.0 MB/s, size: 54.0 KB)
val: Scanning /content/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4/4 117.5it/s 0.0s
val: New cache created: /content/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 5.7it/s 0.2s
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:650: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue

### Train

In [ ]:
#@title Select YOLO11 🚀 logger {run: 'auto'}
logger = 'TensorBoard' #@param ['TensorBoard', 'Weights & Biases']

if logger == 'TensorBoard':
  !yolo settings tensorboard=True
  %load_ext tensorboard
  %tensorboard --logdir .
elif logger == 'Weights & Biases':
  !yolo settings wandb=True